In [1]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import string
from collections import Counter

In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 

In [56]:
digits = pd.read_csv('digit6.csv')
naicscode = pd.read_csv('other_data/NAICS Industry.csv')
uc = pd.read_csv('uscan.csv')
uc = uc.drop('Unnamed: 0', axis = 1)

In [5]:
naics = naicscode.merge(digits,  left_on = 'Sub-Vertical', right_on = 'Industry', how = 'left').rename(columns = \
                                                                                                      {'Industry_x': \
                                                                                                       'Industry'}).drop(\
                                                                                                        'Industry_y', axis = 1)
naics = naics.drop_duplicates().reset_index(drop = True)

In [8]:
#combined three naics categories
combined_naics = naics['Industry'] + ',' + naics['Sub-Vertical']

In [14]:
naics_ind_sub = naics[['Industry', 'Sub-Vertical']]
naics_ind_sub

,Industry,Sub-Vertical
0,Oilseed and Grain Farming,Soybean Farming
1,Oilseed and Grain Farming,Oilseed (except Soybean) Farming
2,Oilseed and Grain Farming,Dry Pea and Bean Farming
3,Oilseed and Grain Farming,Wheat Farming
4,Oilseed and Grain Farming,Corn Farming
...,...,...
1052,Administration of Economic Programs,Regulation of Agricultural Marketing and Commo...
1053,Administration of Economic Programs,"Regulation, Licensing, and Inspection of Misce..."
1054,Space Research and Technology,Space Research and Technology
1055,National Security and International Affairs,National Security


In [17]:
ps = PorterStemmer()
def stemming(ls):
    return [ps.stem(i) for i in ls]

stop_words = set(stopwords.words('english')) 

def combine_matchInd(row):
    if isinstance(row, float): return row
    combined = []
    for i in row:
        if isinstance(i, float): continue
        removePunc = i.replace(',', '').replace('&', '').replace('(', '').replace(')', '').split(' ')
        removeSpace = stemming([x for x in removePunc if x])
        combined += [x for x in removeSpace if x not in stop_words]
    return list(set(combined))

In [19]:
naics_ind_sub2 = naics_ind_sub.copy()
naics_ind_sub2['Industry'] = naics_ind_sub['Industry'].apply(lambda x: x.split(' ')).apply(combine_matchInd)
naics_ind_sub2['Sub-Vertical'] = naics_ind_sub['Sub-Vertical'].apply(lambda x: x.split(' ')).apply(combine_matchInd)

In [21]:
matchInd = pd.read_csv('match_industry.csv').drop('Unnamed: 0', axis = 1)
matchInd.head()

,match,original
0,Traveler Accommodation,Hospitality
1,"Architectural, Engineering, & Related Services",Architecture
2,Management of Companies & Enterprises,Enterprise Resource Planning Software(ERP)
3,Software Publishers,Network Security Hardware & Software
4,Glass & Glass Product Manufacturing,Glass & Gases


In [22]:
matchIndDict = dict(zip(matchInd.original, matchInd.match))

In [66]:
def matchDict(row):
    if isinstance(row, float): return row
    newrow = [x.strip() for x in row]
#     print([matchIndDict[x] for x in newrow])
    return [matchIndDict[x] for x in newrow]

In [63]:
df_ind_split = uc['Industries'].str.replace('and', '&').str.split(',')

In [67]:
matchedInd = df_ind_split.apply(matchDict)

In [68]:
matchedInd

0          [Grantmaking and Giving Services, Business, Pr...
1          [Residential Building Construction, Construction]
2          [Automotive Repair & Maintenance, Consumer Ser...
3          [Scientific Research and Development Services,...
4          [Grantmaking and Giving Services, Business, Pr...
                                 ...                        
1952880    [Grocery and Related Product Merchant Wholesal...
1952881    [Computer Systems Design and Related Services,...
1952882    [Apparel Accessories & Other Apparel Manufactu...
1952883    [Residential Building Construction, Constructi...
1952884                  [Satellite Telecommunications, nan]
Name: Industries, Length: 1952885, dtype: object